In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.2.1'

os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-04-27 21:15:48--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.13MB/s    in 0.2s    

2022-04-27 21:15:48 (5.13 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
wirless_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

In [ ]:
wirless_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

In [ ]:
wirless_df.count()

960872

In [ ]:
review_df = wirless_df.select(["star_rating","helpful_votes", "total_votes", "vine", "verified_purchase"])

In [ ]:
review_df.show()

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          2|            1|          1|   N|                Y|
|          5|            0|          0|   N|                Y|
|          4|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            1|          1|   N|                Y|
|          1|            1|          5|   N|                N|
|          5|            1|          2|   N|                Y|
|          4|            0|          0|   N|                Y|
|          4|            2|          2|   N|                Y|
|          1|            0|          0|   N|                N|
|          3|            0|          0|   N|           

In [ ]:
# Filter for reviews that have a more than 20 total votes
filtered_total_votes = review_df.filter(review_df["total_votes"] >= 20)
filtered_total_votes.count()

9631

In [ ]:
# Filter for a percentage that is greater than 50,60 on helpful votes
# helpful_votes / total_votes > 0.6
percentage_filtered_total_votes = filtered_total_votes.filter(filtered_total_votes["helpful_votes"]/filtered_total_votes["total_votes"] >0.5)
percentage_filtered_total_votes.count()

8373

In [ ]:
# Filter on Vine Reviews
# Descrivbe the results that we habve using stats
vine_reviews  = percentage_filtered_total_votes.filter(percentage_filtered_total_votes["vine"] == "Y")
vine_reviews.describe().show()


+-------+------------------+-----------------+-----------------+----+-----------------+
|summary|       star_rating|    helpful_votes|      total_votes|vine|verified_purchase|
+-------+------------------+-----------------+-----------------+----+-----------------+
|  count|                47|               47|               47|  47|               47|
|   mean|3.9148936170212765|42.61702127659574|47.91489361702128|null|             null|
| stddev|1.0597330926073165|51.83441579201542|55.06629592837206|null|             null|
|    min|                 1|               13|               20|   Y|                N|
|    max|                 5|              349|              370|   Y|                N|
+-------+------------------+-----------------+-----------------+----+-----------------+



In [ ]:
# Filter on Vine Reviews
# Descrivbe the results that we habve using stats
vine_reviews  = percentage_filtered_total_votes.filter(percentage_filtered_total_votes["vine"] == "N")
vine_reviews.describe().show()

+-------+------------------+------------------+-----------------+----+-----------------+
|summary|       star_rating|     helpful_votes|      total_votes|vine|verified_purchase|
+-------+------------------+------------------+-----------------+----+-----------------+
|  count|              8326|              8326|             8326|8326|             8326|
|   mean|3.8017055008407397|42.511890463607976|47.21318760509248|null|             null|
| stddev| 1.523849077118989| 74.11848668726385|80.26867839806437|null|             null|
|    min|                 1|                11|               20|   N|                N|
|    max|                 5|              4004|             4249|   N|                Y|
+-------+------------------+------------------+-----------------+----+-----------------+



In [1]:
# As we can see with the multiple filters that with both vine and non-vine reviews that the mean is 3.915 and 3.800 respectively. With this analysis we could potentially say that 
# vines are not overly effective compared to non-vines. Also there are more data points with non-vines are therefore we can see more reliablity in the results.   